In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
!pip install transformers datasets evaluate scikit-learn tqdm

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/a2/e7/cbca9e2d2590eb9b5aa8f7ebabe1beb1498f9462d2ecede5c9fd9735faaf/evaluate-0.4.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/84.0 kB ? eta -:--:--
   --------- ------------------------------ 20.5/84.0 kB 640.0 kB/s eta 0:00:01
   ---------------------------------------- 84.0/84.0 kB 1.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.metrics import classification_report
import numpy as np
import evaluate

In [4]:
df = pd.read_csv("data/combined_dataset_forbehandlad.csv")
df = df[["text", "label"]].dropna().reset_index(drop=True)

In [5]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)


In [6]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True)

In [8]:
dataset = dataset.map(tokenize, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/844 [00:00<?, ? examples/s]

Map:   0%|          | 0/212 [00:00<?, ? examples/s]

In [9]:
metric = evaluate.load("accuracy")

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = metric.compute(predictions=predictions, references=labels)
    return acc

In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [12]:
training_args = TrainingArguments(
    output_dir="./bert_results",
    # The parameter has been replaced with evaluation_strategy="epoch"
    eval_steps = 100, # Evaluate every 100 steps.
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",

)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\mirac\AppData\Local\Temp\ipykernel_20240\3647305547.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()


c:\Users\mirac\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=318, training_loss=0.29912605525562597, metrics={'train_runtime': 162.9751, 'train_samples_per_second': 15.536, 'train_steps_per_second': 1.951, 'total_flos': 13784888672640.0, 'train_loss': 0.29912605525562597, 'epoch': 3.0})

In [15]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Spara modellen
model.save_pretrained('model/bert_model')

# Spara tokenizern
tokenizer.save_pretrained('model/bert_model')


('model/bert_model\\tokenizer_config.json',
 'model/bert_model\\special_tokens_map.json',
 'model/bert_model\\vocab.txt',
 'model/bert_model\\added_tokens.json',
 'model/bert_model\\tokenizer.json')

In [16]:
predictions = trainer.predict(dataset["test"])
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=-1)

In [17]:
print("=== Klassificeringsrapport ===")
print(classification_report(y_true, y_pred, digits=3))

=== Klassificeringsrapport ===
              precision    recall  f1-score   support

           0      0.881     0.851     0.865        87
           1      0.898     0.920     0.909       125

    accuracy                          0.892       212
   macro avg      0.890     0.885     0.887       212
weighted avg      0.891     0.892     0.891       212



In [21]:
print(df['source'].value_counts())



KeyError: 'source'